In [ ]:
# Загрузка библиотеки OpenAI-Whisper
! pip install git+https://github.com/openai/whisper.git
! pip install jiwer

In [ ]:
import os
import numpy as np
import torch
import pandas as pd
import whisper

# Для создания прогресс-баров
from tqdm.notebook import tqdm 

# Переключение на GPU, если GPU доступно
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Загрузка модели Whisper Large
model = whisper.load_model("large")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters.")

# Настройка опций: язык — русский, временные метки — нужны.
options = whisper.DecodingOptions(language="ru", without_timestamps=False)

In [ ]:
from whisper.utils import get_writer
from text_processing import text_preprocess

# Загружаем и предобрабатываем prompt из файла
prompt_transcribe = text_preprocess('prompts/lemur.txt')

# Транскрибируем аудио с помощью Whisper
result = model.transcribe("grishunin_35m1s_40m20s.mp3", # Файл для обработки
                          initial_prompt=prompt_transcribe, # Входной prompt для fine-tuning'а словаря
                          verbose=True, # Выводим результаты на экран сразу по ходу работы
                          condition_on_previous_text=False # Распознаваемый текст НЕ воспринимаем как продолжение prompt
                          )

# Задаём текущую папку и сохраняем субтитры в формате SRT (субтитры)
writer = get_writer("srt", ".")  
writer(result, "audio_file_name", {})